In [6]:
import numpy as np
import copy
import json
import math
from glob import glob
import scipy.spatial.distance as sciDist
from tqdm import tqdm
import requests
import time
import itertools
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import islice
from PIL import Image
import re
from tqdm import tqdm


# Headless simulator version
index = 7 # local server index 
#API_ENDPOINT = 'http://localhost:400' + str(index) + '/simulation' # NOT THE LS VERSION
API_ENDPOINT = 'http://localhost:400' + str(1) + '/simulation' # NOT THE LS VERSION
HEADERS = {"Content-Type": "application/json"}
batchCount = 25 # Send this number of samples to MotionGen each time 
speedscale = 1
steps = 360
minsteps = int(steps*20/360)

# Search for "stephenson" or "watt" in the JSON file and collect their names and indices
json_path = "KV_468_062324.json"
with open(json_path, "r") as f:
    data = json.load(f)

matches = []
for idx, name in enumerate(data):
    lname = name.lower()
    if "step" in lname or "watt" in lname:
        matches.append([name, idx])
print(matches)


# Things for 4 bar 
mechType = index
types = ['RRRR', 'RRRP', 'RRPR', 'PRPR']
for mechanism in matches:
    types.append(mechanism[0])
print(types)
# ['RRRR', 'RRRP', 'RRPR', 'PRPR', 'Watt2T1A1', 'Steph1T3', 'Watt1T1A1', 'Steph3T2A2', 'Watt1T3A2', 'Steph2T2A1', 'Steph3T1A2', 'Watt2T2A2', 'Watt1T2A1', 'Steph2T1A1', 'Steph3T2A1', 'Watt1T1A2', 'Watt2T1A2', 'Steph1T2', 'Steph1T1', 'Watt1T3A1', 'Watt1T2A2', 'Watt2T2A1', 'Steph3T1A1', 'Steph2T1A2', 'Steph2T2A2']

# Create a directory in outputs-6bar for each type in types
'''
for t in types:
    if t in ('RRRR', 'RRRP', 'RRPR', 'PRPR'):
        continue
    dir_path = os.path.abspath("./outputs-6bar/" + t)
    os.makedirs(dir_path, exist_ok=True)'''



typeIndex = [49, 64, 155, 175] # to avoid confusion from any other type
for mechanism in matches:
    typeIndex.append(mechanism[1])

print(typeIndex)
# [49, 64, 155, 175, 0, 2, 19, 26, 37, 39, 73, 82, 90, 110, 111, 125, 133, 135, 137, 139, 149, 163, 169, 176, 177]


#couplerCurveIndex = [4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


#confirm that all c values are 1 at index 7 for 6 bars

bsi = "BSIdict_468_062324_3.json"
with open(bsi, "r") as b:
    data = json.load(b)

for key in types:
    if key in data:
        c_value = (data)[key].get("c")
        print(key, c_value)

# Update couplerCurveIndex so that for each type, its value is the index of the number 1 in the c_value list from BSIdict
couplerCurveIndex = []
for key in types:
    if key in data:
        c_value = data[key].get("c")
        if isinstance(c_value, list) and 1 in c_value:
            couplerCurveIndex.append(c_value.index(1))
        else:
            couplerCurveIndex.append(4)  # fallback default
    else:
        couplerCurveIndex.append(4)      # fallback default
print("Updated couplerCurveIndex:", couplerCurveIndex)
# Updated couplerCurveIndex: [4, 4, 4, 4, 7, 7, 7, 7, 6, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 4, 7, 4]


savePointNumber = [5, 6, 7, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8] #len of c arrat acording to BSIdict
needAddtional = [False, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False] # notice that when true, some of the points can change its position randomly 
#initStates = np.load("./npy-inputs/" + 'RandPos-.npy')

initStates = np.load("/gpfs/scratch/raytang/walking-dataset-generator-old-simulator/Image-method-Synthesis-main/npy-inputs/RandPos-.npy")
errCtr = 0
batch = []
batchSaveStr = []
batchSaveNpyStr = []


# The transformation 
#np.save(saveDir + name + ' ' + types[index], param)
saveDir = os.path.abspath("./outputs-6bar/" + types[index] )
saveDirNpy = os.path.abspath("./outputs-6bar/" + types[index] + "-npy")
print(saveDir, saveDirNpy)



[['Watt2T1A1', 0], ['Steph1T3', 2], ['Watt1T1A1', 19], ['Steph3T2A2', 26], ['Watt1T3A2', 37], ['Steph2T2A1', 39], ['Steph3T1A2', 73], ['Watt2T2A2', 82], ['Watt1T2A1', 90], ['Steph2T1A1', 110], ['Steph3T2A1', 111], ['Watt1T1A2', 125], ['Watt2T1A2', 133], ['Steph1T2', 135], ['Steph1T1', 137], ['Watt1T3A1', 139], ['Watt1T2A2', 149], ['Watt2T2A1', 163], ['Steph3T1A1', 169], ['Steph2T1A2', 176], ['Steph2T2A2', 177]]
['RRRR', 'RRRP', 'RRPR', 'PRPR', 'Watt2T1A1', 'Steph1T3', 'Watt1T1A1', 'Steph3T2A2', 'Watt1T3A2', 'Steph2T2A1', 'Steph3T1A2', 'Watt2T2A2', 'Watt1T2A1', 'Steph2T1A1', 'Steph3T2A1', 'Watt1T1A2', 'Watt2T1A2', 'Steph1T2', 'Steph1T1', 'Watt1T3A1', 'Watt1T2A2', 'Watt2T2A1', 'Steph3T1A1', 'Steph2T1A2', 'Steph2T2A2']
[49, 64, 155, 175, 0, 2, 19, 26, 37, 39, 73, 82, 90, 110, 111, 125, 133, 135, 137, 139, 149, 163, 169, 176, 177]
RRRR [0, 0, 0, 0, 1]
RRRP [0, 0, 0, 0, 1, 0]
RRPR [0, 0, 0, 0, 1, 0, 0]
PRPR [0, 0, 0, 0, 1]
Watt2T1A1 [0, 0, 0, 0, 0, 0, 0, 1]
Steph1T3 [0, 0, 0, 0, 0, 0, 0, 1]

In [7]:
# good old ones 

def isValid(seq):
    if len(seq.shape) == 2:
        isVal = np.var(seq[:,0]) <= 5e-3 and np.var(seq[:,1]) <= 5e-3
    else:
        isVal = len(seq) == 0 or np.var(seq) <= 5e-3

    if isVal:
        return False
    else:
        return True


def get_pca_inclination(qx, qy, ax=None, label=''):
    """ Performs the PCA
        Return transformation matrix
    """
    cx = np.mean(qx)
    cy = np.mean(qy)
    covar_xx = np.sum((qx - cx)*(qx - cx))/len(qx)
    covar_xy = np.sum((qx - cx)*(qy - cy))/len(qx)
    covar_yx = np.sum((qy - cy)*(qx - cx))/len(qx)
    covar_yy = np.sum((qy - cy)*(qy - cy))/len(qx)
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])
    eig_val, eig_vec= np.linalg.eig(covar)

    # Inclination of major principal axis w.r.t. x axis
    if eig_val[0] > eig_val[1]:
        phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
    else:
        phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])

    return phi


def get_normalize_curve(jd, steps=None, rotations=1, normalize=True, transformParas=None):
    jd = np.array(jd)
    joint_data_n, x_mean, y_mean, denom, phi = [], None, None, None, None
    if isValid(jd):
        if steps:
            sample_indices = np.linspace(0, jd.shape[0]-1, steps, dtype=np.int32)
            jd = jd[sample_indices,:]
        if normalize:
            if not transformParas:
                x_mean = np.mean(jd[:,0], axis=0, keepdims=True)
                y_mean = np.mean(jd[:,1], axis=0, keepdims=True)
            else:
                x_mean, y_mean, denom, phi = transformParas
            jd[:,0] = jd[:,0] - x_mean
            jd[:,1] = jd[:,1] - y_mean

            if not transformParas:
                denom = np.sqrt(np.var(jd[:,0], axis=0, keepdims=True) + np.var(jd[:,1], axis=0, keepdims=True))
                denom = np.expand_dims(denom, axis=1)
            jd = jd / denom
            t = 0
        if not transformParas:
            phi = -get_pca_inclination(jd[:,0], jd[:,1])
        jd[:,0], jd[:, 1] = rotate_curve(jd, phi)
        for tt in range(rotations):
            joint_data_n.append(jd.copy())
            if rotations > 1:
                jd[:,0], jd[:,1] = rotate_curve(jd, t)
                t = 2*np.pi/rotations

    return joint_data_n, x_mean, y_mean, denom, phi


def rotate_curve(cur, theta):
    cpx = cur[:,0]*np.cos(theta) - cur[:,1]*np.sin(theta)
    cpy = cur[:,0]*np.sin(theta) + cur[:,1]*np.cos(theta)
    return cpx, cpy


def digitize_seq(nums, minlim, maxlim, bin_size=64):
    bins = np.linspace(minlim, maxlim, bin_size-1)
    nums_indices = np.digitize(nums, bins)
    return nums_indices


def get_normalize_joint_data_wrt_one_curve(joint_data, ref_ind = 4):
    ''' input s = [num_curves, num_points, 2]
    '''
    joint_data_n = []
    s = np.array(joint_data)
    if isValid(s[ref_ind]):
        x_mean = np.mean(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True)
        y_mean = np.mean(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True)
        s[:,:,0] = s[:,:,0] - x_mean
        s[:,:,1] = s[:,:,1] - y_mean
        denom = np.sqrt(np.var(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True) + np.var(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True))
        denom = np.expand_dims(denom, axis=2) #is this scale? 
        s = s / denom
        phi = -get_pca_inclination(s[ref_ind:ref_ind+1,:,0], s[ref_ind:ref_ind+1,:,1])
        for i in range(s.shape[0]):
            s[i,:,0], s[i,:,1] = rotate_curve(s[i], phi)
    else:
        return s, [None, None, None, None], False

    # s has a shape of (j_num, state, dim)
    return s, [x_mean[0][0], y_mean[0][0], denom[0][0][0], phi], True # tx, ty, scaling, rotation angle 


##############################################################################################
# There are some other necessary transformations. (x_mean, y_mean, phi, denom) are from get_normalize_curve. 
##############################################################################################
def get_image_from_point_cloud(points, xylim, im_size, inverted = True, label=None):
    mat = np.zeros((im_size, im_size, 1), dtype=np.uint8)
    x = digitize_seq(points[:,0], -xylim, xylim, im_size)
    if inverted:
        y = digitize_seq(points[:,1]*-1, -xylim, xylim, im_size)
        mat[y, x, 0] = 1
    else:
        y = digitize_seq(points[:,1], -xylim, xylim, im_size)
        mat[x, y, 0] = 1
    return mat


def process_mech_102723(jointData, ref_ind, im_size = 64, xylim = 3.5, inverted = True, swapAxes = True):
    paras = None

    # It is possible the jointData format is (angles, joint, (x, y)). 
    # You should put a True if this happens. (This is how files are saved).
    # I literally don't understand why I saved jointData with a shape of (angles, joint, (x, y)) 
    if swapAxes:
        jointData = np.swapaxes(jointData, 0, 1)

    # This converts all 
    jointData, paras, success = get_normalize_joint_data_wrt_one_curve(jointData, ref_ind= ref_ind)

    # jointData format from now on becomes np.array with a shape of (joint, curve_length, dimension)
    jointData = np.array(jointData)

    if success:
        # get binaryImage 
        jd = jointData[ref_ind]
        mat = get_image_from_point_cloud(jd, xylim=xylim, im_size=im_size, inverted=inverted)
        return mat, paras, success
    else: 
        return None, None, success


def calc_dist(coord):
    # Calculate differences using broadcasting
    diffs = coord[:, np.newaxis, :] - coord[np.newaxis, :, :]
    squared_dists = np.sum(diffs ** 2, axis=2)

    # Extract the upper triangle indices where i < j
    i, j = np.triu_indices(len(coord), k=1)
    dist_arr = np.sqrt(squared_dists[i, j])
    dist_arr = dist_arr/min(dist_arr)
    return np.round(dist_arr, 2)

In [8]:
############################################################################################## 
# An even better version of normalization method (it can be slower sometimes)
# Uses reflection correction (contributed by the paper Geometric Invariant Curve and Surface Normalization)  
# Thanks to Zhijie who noticed these two can be used together. 
##############################################################################################

def rotate_curve(curve, phi):
    infunction_scale = 100
    # curve is scaled 100 times for numerical accuracy 
    # incoming curve shape: (n,2)
    x = curve[:, 0] * infunction_scale
    y = curve[:, 1] * infunction_scale
    # Compute the rotated coordinates
    x_rotated = x * np.cos(phi) - y * np.sin(phi)
    y_rotated = x * np.sin(phi) + y * np.cos(phi)
    # Combine the rotated coordinates into a new curve
    rotated_curve = np.column_stack((x_rotated, y_rotated))
    return rotated_curve/infunction_scale


def center_data(X):
    """ Center the data by subtracting the mean of each column.
        Return the translated X and the translation matrix 
    """
    m = np.mean(X, axis=0) # (n, 2)
    return X - m, np.matrix([[1, 0, -m[0]], [0, 1, -m[1]], [0, 0, 1]]) # equal to XP this is a translation matrix tranposed  


def scale_data(X, scaling = 0): 
    """ Scale the data according to two different metrics 
        If scaling == 0 (default), scaling method is normalization (average distance 1)
        If otherwise, scaling method is standardization to a certain scale 
        Return the scaled X, and the scaling matrix. 
    """
    if scaling == 0:
        # use variance. 
        denom = np.sqrt(np.var(X[:,0]) + np.var(X[:,1]))
        scaled_curve = X /denom
        ScaleMat = np.matrix([[1/denom, 0, 0], [0, 1/denom, 0], [0, 0, 1]])
    else:
        # Compute the maximum distance from the origin 
        max_distance = np.max(np.linalg.norm(X, axis=1))
        scaled_curve = X * scaling / max_distance
        ScaleMat = np.matrix([[scaling/max_distance, 0, 0], [0, scaling/max_distance, 0], [0, 0, 1]])
    return scaled_curve, ScaleMat


def rotate_data(X, tol = 1e-4, randinit = False): 
    """ Performs the PCA and determines rotation angle phi 
        More precisely it is snapping the greatest principal axis to the X-axis. 
        Return the rotated X and the rotation matrix 
    """
    phiInit = 0
    if randinit:
        phiInit = np.random.rand() * math.pi * 2 

    rotationMatInit = np.matrix([
        [np.cos(phiInit), -np.sin(phiInit), 0], 
        [np.sin(phiInit), np.cos(phiInit), 0],
        [0, 0, 1] 
    ])

    X0 = rotate_curve(X, phiInit)
    cx = np.mean(X0[:,0])
    cy = np.mean(X0[:,1])
    covar_xx = np.sum((X0[:,0] - cx)*(X0[:,0] - cx))/X0.shape[0]
    covar_xy = np.sum((X0[:,0] - cx)*(X0[:,1] - cy))/X0.shape[0]
    covar_yx = np.sum((X0[:,1] - cy)*(X0[:,0] - cx))/X0.shape[0]
    covar_yy = np.sum((X0[:,1] - cy)*(X0[:,1] - cy))/X0.shape[0]
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])

    if np.abs(np.linalg.det(covar)) < tol:
        phi = 0 # why rotate anyway? 
    else:
        eig_val, eig_vec= np.linalg.eig(covar) 
        # Inclination of major principal axis w.r.t. x axis
        # Enforcing the cross-product of the two eigenvectors to be greater than 0. 
        # Not necessary, but it looks clean to do so. 
        # Eigenvector matrix: [a, b], det = crossproduct of b x a
        if np.linalg.det(eig_vec) > 0:
            eig_vec[0,:] = -eig_vec[0,:] # enforcing a x b > 0 
        if eig_val[0] > eig_val[1]:
            phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
        else:
            phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])
    rotated_curve = rotate_curve(X0, phi)
    rotationMat = np.matrix([
        [np.cos(phi), -np.sin(phi), 0], 
        [np.sin(phi), np.cos(phi), 0],
        [0, 0, 1] 
    ])

    return rotated_curve, np.matmul(rotationMat, rotationMatInit)


def reflect_data(X):
    """ Computes the third order moment and determines the reflections 
        The data must be rotated before this step. 

    """
    # Reflection normalization 
    x_scaled = X[:, 0]
    y_scaled = X[:, 1]

    # see paper Geometric Invariant Curve and Surface Normalization
    # compute the 3rd-order moments 
    m12 = np.sum((x_scaled**1)*(y_scaled**2))
    m21 = np.sum((x_scaled**2)*(y_scaled**1))
    signm12 = np.sign(m12)
    signm21 = np.sign(m21)
    if np.abs(signm12) < 1e-5:
        signm12 = 1
    if np.abs(signm21) < 1e-5:
        signm21 = 1

    reflectionMat = np.array(
        [[signm12, 0],
         [0, signm21]]
    ) 

    if np.abs(m12) > np.abs(m21):
        reflectionMat = np.matmul(np.array([[0,1],[1,0]]), reflectionMat)

    reflected_Curve = np.matmul(reflectionMat, np.array(X).T).T
    reflectionMat = np.matrix(
        [[reflectionMat[0,0], reflectionMat[0,1], 0], 
         [reflectionMat[1,0], reflectionMat[1,1], 0], 
         [0, 0, 1]
        ]
    ) 

    return reflected_Curve, reflectionMat


# Digitization 
def digitize_seq(nums, minlim, maxlim, bin_size=64):
    bins = np.linspace(minlim, maxlim, bin_size-1)
    nums_indices = np.digitize(nums, bins)
    return nums_indices


def get_image_from_point_cloud(points, xylim, im_size, inverted = True, label=None):
    mat = np.zeros((im_size, im_size, 1), dtype=np.uint8)
    x = digitize_seq(points[:,0], -xylim, xylim, im_size)
    if inverted:
        y = digitize_seq(points[:,1]*-1, -xylim, xylim, im_size)
        mat[y, x, 0] = 1
    else:
        y = digitize_seq(points[:,1], -xylim, xylim, im_size)
        mat[x, y, 0] = 1
    return mat


# Returns normalized curve, Transformation matrix, and Success (if determinant is greater than tolerance for numerical stability)
def normalize_data_122223(X, scaling = 0, tol = 1e-8, maxiter = 2):
    X1, M1 = center_data(X) 
    X1, M2 = scale_data(X1, scaling = scaling)
    X1, M3 = rotate_data(X1)
    X1, M4 = reflect_data(X1)
    M = M4*M3*M2*M1 # This is the transformation matrix 

    detVal = np.abs(np.linalg.det(M))
    if detVal*scaling < tol:
        for i in range(maxiter):
            X1, M1 = center_data(X1)
            X1, M2 = scale_data(X1, scaling = scaling)
            X1, M3 = rotate_data(X1, randinit= True)
            X1, M4 = reflect_data(X1)
            if np.abs(np.linalg.det(M)) > tol or detVal*10 < np.abs(np.linalg.det(M)):
                break
    return X1, M4*M3*M2*M1, np.abs(np.linalg.det(M)) > tol


def matmul_jd(jd, mat):
    # input should be (..., ,2)
    # the operation: 
    jd = np.array(jd)
    oldshape = jd.shape
    njd= np.reshape(jd, (-1, 2))
    hc1= np.ones((njd.shape[0], 1))
    njd=np.matrix(np.concatenate([njd, hc1], axis = 1)).transpose()
    njd=np.array((mat*njd).transpose())[:,0:2].reshape(oldshape)
    return njd


# Processing 
# THIS IS THE MAIN FUNCTION YOU NEED (second version) 
# Input is the mechanism and coupler point index (ref_ind), Output is matImg, [<two transformation matrix instead of six parameters>], and Success. 
# NOTICE 1: the parameter has SIX parameters, not four (x, y translation, angle and scale). The addtional parameters are reflectionMat, seqReversed (refrlection matrix and whether or not the sequence is reverse in order)
# NOTICE 2: the transformation matrices are numpy arrays. You need to convert them to lists before converting them into .json format objects. 
# I suggest you save the a, b, c, d, e, f values in the matrix. 
# A transformation matrix is shaped as: 
# [[a, c, e]
#  [b, d, f]
#  [0, 0, 1]]
def process_mech_051524(jointData, ref_ind, im_size = 64, xylim = 3.5, inverted = True):
    # New JD shape is always (states, joints, dimensions)
    paras = None
    # get matrices according to curve. 
    nc, mat, success = normalize_data_122223(jointData[:,ref_ind,:], scaling = 3.5)
    NR_MG = mat # transform original position to normalized position # NR_MG * jointData = normalized joint data
    MG_NR = np.linalg.inv(NR_MG) 

    if success:
        # get binaryImage # rohit's choice on what to save 
        paras = NR_MG
        matImg = get_image_from_point_cloud(nc, xylim=xylim, im_size=im_size, inverted=inverted)
        return matImg, paras, success
    else: 
        return None, None, success

    
def calc_dist(coord):
    # Calculate differences using broadcasting
    diffs = coord[:, np.newaxis, :] - coord[np.newaxis, :, :]
    squared_dists = np.sum(diffs ** 2, axis=2)

    # Extract the upper triangle indices where i < j
    i, j = np.triu_indices(len(coord), k=1)
    dist_arr = np.sqrt(squared_dists[i, j])
    dist_arr = dist_arr/min(dist_arr)
    return np.round(dist_arr, 2)

In [9]:
def format_floats_to_string(float_list):
    formatted_strings = [f"{value:.3f}" for value in float_list]
    result_string = ' '.join(formatted_strings)
    return ' ' + result_string

# Example usage
float_values = [3920, -1.000e-03, 5.393e+00, 1.748e+00, -8.786e+00, 2.079e+00, 5.023e+00, -4.854e+00, 4.230e-01, 5.406e+00, -1.882e+00, -1.388e+00, 0.241, -0.285, 0.455, 0.899, 0.0, 1.0]

formatted_string = format_floats_to_string(float_values)
print(formatted_string)



 3920.000 -0.001 5.393 1.748 -8.786 2.079 5.023 -4.854 0.423 5.406 -1.882 -1.388 0.241 -0.285 0.455 0.899 0.000 1.000


In [10]:
normFailCtr = 0

mechType = 0
# ['RRRR', 'RRRP', 'RRPR', 'PRPR', 'Watt2T1A1', 'Steph1T3', 'Watt1T1A1', 'Steph3T2A2', 'Watt1T3A2', 'Steph2T2A1', 'Steph3T1A2', 'Watt2T2A2', 'Watt1T2A1', 'Steph2T1A1', 'Steph3T2A1', 'Watt1T1A2', 'Watt2T1A2', 'Steph1T2', 'Steph1T1', 'Watt1T3A1', 'Watt1T2A2', 'Watt2T2A1', 'Steph3T1A1', 'Steph2T1A2', 'Steph2T2A2']
saveDir = os.path.abspath("./outputs-6bar/" + types[mechType] )
saveDirNpy = os.path.abspath("./outputs-6bar/" + types[mechType] + "-npy")

NUM_MECHS = 500000
#mechtype=6

#saveDir = os.path.abspath("./outputs-6bar/" + types[mechType] )
#
# Prefer Savitzky-Golay if available
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter1d

def has_sharp_edges(coords, curvature_threshold=15.0, smoothing_sigma=1.0):
    """
    Detect if a 2D curve has sharp edges using curvature.

    Parameters:
    - coords: np.array of shape (N,2) representing the curve points.
    - curvature_threshold: float, curvature value above which a point is considered a sharp corner.
    - smoothing_sigma: float, optional Gaussian smoothing to reduce numerical noise.

    Returns:
    - bool: True if curve has sharp edges, False otherwise
    - curvature: np.array of curvature values along the curve (optional, useful for plotting)
    """
    if smoothing_sigma > 0:
        x = gaussian_filter1d(coords[:,0], sigma=smoothing_sigma)
        y = gaussian_filter1d(coords[:,1], sigma=smoothing_sigma)
    else:
        x = coords[:,0]
        y = coords[:,1]

    dx = np.gradient(x)
    dy = np.gradient(y)
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)

    curvature = np.abs(dx * ddy - dy * ddx) / (dx**2 + dy**2)**1.5

    return np.any(curvature > curvature_threshold), curvature


def resample_uniform(points, M):
    # points: (N,2) numpy
    # Return M points uniformly along arc length
    dif = np.diff(points, axis=0)
    seglen = np.sqrt((dif**2).sum(axis=1))
    s = np.concatenate(([0], np.cumsum(seglen)))
    total = s[-1]
    if total == 0:
        return np.repeat(points[:1], M, axis=0)
    s_uniform = np.linspace(0, total, M)
    x = np.interp(s_uniform, s, points[:,0])
    y = np.interp(s_uniform, s, points[:,1])
    return np.column_stack((x, y))


def pca_align(points):
    # points: (N,2) numpy array (assumes centered or will center here)
    pts = points - np.mean(points, axis=0)
    # Compute principal components via SVD
    U, S, VT = np.linalg.svd(pts, full_matrices=False)
    pcs = VT.T  # columns are principal directions
    # rotate so first PC aligns with x-axis
    rotated = pts @ pcs
    return rotated, pcs  # rotated coords, principal direction basis
# eps slope used to be 0.0015



def detect_flat_via_pca(points, K_keep=12, N_resample=512, eps_slope=0.0075, min_fraction=1/4):
    # points: original coupler coords
    pts = resample_uniform(points, N_resample)
    rotated, pcs = pca_align(pts)
    # rotated[:,1] is the coordinate perpendicular to main axis (height)
    dy = np.gradient(rotated[:,1])
    flat_mask = np.abs(dy) < eps_slope
    # find the longest contiguous run
    from itertools import groupby
    runs = [sum(1 for _ in g) for val,g in groupby(flat_mask) if val]
    max_run = max(runs) if runs else 0
    flat_fraction = max_run / len(flat_mask)
    is_flat_enough = flat_fraction >= min_fraction
    # orientation: angle of first PC relative to x-axis
    # pcs[:,0] is first PC in original frame
    first_pc = pcs[:,0]
    angle = np.arctan2(first_pc[1], first_pc[0])  # radians

    max_run = 0
    run_indices = []
    current_run = []

    for i, is_flat in enumerate(flat_mask):
        if is_flat:
            current_run.append(i)
            if len(current_run) > max_run:
                max_run = len(current_run)
                run_indices = current_run.copy()
        else:
            current_run = []

    return {
        'flat_fraction': float(flat_fraction),
        'is_flat_enough': bool(is_flat_enough),
        'orientation_rad': float(angle),
        'orientation_deg': float(np.degrees(angle)),
        'max_run': int(max_run),
        'N': int(len(flat_mask)),
        'flat_indices': np.array(run_indices, dtype=int), 
        'pts': pts,
    }


def ground_clearance_check(path_coords, joint_coords, flat_indices):
    if len(flat_indices) == 0:
        print("No flat indices provided.")
        return False
    flat_pts = path_coords[flat_indices]
    # Fit line to flat portion
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)

    # Create line values across full x-range
    x_line = np.linspace(path_coords[:,0].min(), path_coords[:,0].max(), 200)
    y_line = m * x_line + b

    # Signed distance of path points to the line

    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)

    majority_sign = np.sign(np.sum(d_path))

    if majority_sign == 0:

        majority_sign = 1  # fallback if perfectly balanced



    # Signed distance for joints

    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)



   
    # Debug print

    all_clear = np.all(np.sign(d_joints) == majority_sign)

    return all_clear


def plot_flat_fit_with_joints(path_coords, flat_indices, joint_coords, title="Flat Fit Check"):
    """
    Plot the full curve, flat segment, best-fit line, and joint coordinates
    with indices labeled.
    """
    if len(flat_indices) == 0:
        print("No flat indices provided.")
        return
    
    flat_pts = path_coords[flat_indices]

    # Fit line to flat portion
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)

    # Create line values across full x-range
    x_line = np.linspace(path_coords[:,0].min(), path_coords[:,0].max(), 200)
    y_line = m * x_line + b

    # Signed distance of path points to the line

    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)

    majority_sign = np.sign(np.sum(d_path))

    if majority_sign == 0:

        majority_sign = 1  # fallback if perfectly balanced



    # Signed distance for joints

    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)



    # Plot original curve
    plt.figure(figsize=(7,7))
    plt.plot(path_coords[:,0], path_coords[:,1], 'b-', label='Full curve')
    plt.plot(flat_pts[:,0], flat_pts[:,1], 'ro', label='Flat segment')
    plt.plot(x_line, y_line, 'k--', label='Best fit line (flat)')

    # Plot joints with colors depending on clearance

    for i, (x, y) in enumerate(joint_coords):
        if np.sign(d_joints[i]) == majority_sign:
            plt.scatter(x, y, c='g', marker='x', s=80)  # green if correct side
        else:
            plt.scatter(x, y, c='r', marker='x', s=80)  # red if wrong side
        plt.text(x, y, str(i), fontsize=9, ha='right', va='bottom')

    plt.axis('equal')
    plt.legend()
    plt.title(title)
    plt.show()

    # Debug print

    all_clear = np.all(np.sign(d_joints) == majority_sign)

    print(f"Line slope m = {m:.4f}, intercept b = {b:.4f}")

    print(f"Majority sign = {majority_sign}")

    print(f"Joint distances = {d_joints}")

    print(f"Ground clearance OK? {all_clear}")

def save_flat_fit_with_joints(path_coords, flat_indices, joint_coords, save_dir, filename):
    """
    Same as plot_flat_fit_with_joints, but saves the plot to a directory instead of displaying or printing.
    """

    if len(flat_indices) == 0:
        return
    
    flat_pts = path_coords[flat_indices]

    # Fit line to flat portion
    m, b = np.polyfit(flat_pts[:,0], flat_pts[:,1], 1)

    # Create line values across full x-range
    x_line = np.linspace(path_coords[:,0].min(), path_coords[:,0].max(), 200)
    y_line = m * x_line + b

    # Signed distance of path points to the line
    d_path = path_coords[:,1] - (m*path_coords[:,0] + b)
    majority_sign = np.sign(np.sum(d_path))
    if majority_sign == 0:
        majority_sign = 1  # fallback if perfectly balanced

    # Signed distance for joints
    d_joints = joint_coords[:,1] - (m*joint_coords[:,0] + b)

    # Plot original curve
    plt.figure(figsize=(7,7))
    plt.plot(path_coords[:,0], path_coords[:,1], 'b-', label='Full curve')
    plt.plot(flat_pts[:,0], flat_pts[:,1], 'ro', label='Flat segment')
    plt.plot(x_line, y_line, 'k--', label='Best fit line (flat)')

    # Plot joints with colors depending on clearance
    for i, (x, y) in enumerate(joint_coords):
        if np.sign(d_joints[i]) == majority_sign:
            plt.scatter(x, y, c='g', marker='x', s=80)  # green if correct side
        else:
            plt.scatter(x, y, c='r', marker='x', s=80)  # red if wrong side
        plt.text(x, y, str(i), fontsize=9, ha='right', va='bottom')

    plt.axis('equal')
    plt.legend()
    plt.title(f"coords:{filename}")

    # Ensure save_dir exists
    os.makedirs(save_dir, exist_ok=True)
    # Generate a filename
    
    filename = f"{filename}.png"
    filepath = os.path.join(save_dir, filename)
    plt.savefig(filepath)
    plt.close()

from shapely.geometry import LineString

def has_self_intersections(coords):
    line = LineString(coords)
    return not line.is_simple  # False if self-crossing



def is_closed(pts):
    # Check if path is open or closed
    start_pt = pts[0]
    end_pt = pts[-1]
    path_dist = np.linalg.norm(end_pt - start_pt)
    is_closed = path_dist < 0.1
    return is_closed

def str_to_coords(s):
    s = str(s)
    nums = [float(x) for x in s.strip().split()]
    return [[nums[j], nums[j+1]] for j in range(0, len(nums), 2)]

def is_walking(coords):

    pca_output = detect_flat_via_pca(normalized_coords)

    sharp, kappa = has_sharp_edges(normalized_coords, curvature_threshold=150.0)

    if pca_output['is_flat_enough'] and is_closed(normalized_coords) and (not sharp):
        return True
    else:
        return False

numCoords = 5 # the number of joints, for 4 bars - 5 joints
#             0         1       2       3      4             5           6             7            8          9              10            11            12            13            14           15           16           17            18         19          20           21           22            23            24            
# mechtypes: ['RRRR', 'RRRP', 'RRPR', 'PRPR', 'Watt2T1A1', 'Steph1T3', 'Watt1T1A1', 'Steph3T2A2', 'Watt1T3A2', 'Steph2T2A1', 'Steph3T1A2', 'Watt2T2A2', 'Watt1T2A1', 'Steph2T1A1', 'Steph3T2A1', 'Watt1T1A2', 'Watt2T1A2', 'Steph1T2', 'Steph1T1', 'Watt1T3A1', 'Watt1T2A2', 'Watt2T2A1', 'Steph3T1A1', 'Steph2T1A2', 'Steph2T2A2'] (4 and 6 bars)
for index in range(0, 25): # change this to range(0, 4) for 4-bar and range(5, 25) for 6-bar
    mechType = index
    print(types[mechType])
    if mechType <= 3:
        saveDir = os.path.abspath("./outputs-4bar/" + types[mechType] )
        saveDirNpy = os.path.abspath("./outputs-4bar/" + types[mechType] + "-npy")
        saveDirWalking = os.path.abspath("./outputs-4bar/walking_" + types[mechType])
        saveDirWalkingPlot = os.path.abspath("./outputs-4bar/walking_plot_" + types[mechType])

        numCoords = 5
    else:
        saveDir = os.path.abspath("./outputs-6bar/" + types[mechType] )
        saveDirNpy = os.path.abspath("./outputs-6bar/" + types[mechType] + "-npy")
        saveDirWalking = os.path.abspath("./outputs-6bar/walking_" + types[mechType])
        saveDirWalkingPlot = os.path.abspath("./outputs-6bar/walking_plot_" + types[mechType])

        numCoords = 8

    if mechType in [8, 9, 10, 19, 22, 24]: # some types of 6 bar only have 7 joints
        numCoords = 7

    # Create directories if they do not exist
    for d in [saveDir, saveDirNpy, saveDirWalking, saveDirWalkingPlot]:
        os.makedirs(d, exist_ok=True)

    
    
    for initState in tqdm(initStates[mechType*NUM_MECHS: (mechType+1)*NUM_MECHS, :numCoords*2]): # 5*2 for 4 bars, 8*2 for 6 bars
        coord = np.round(initState, 3).reshape((numCoords,2))
        dist = calc_dist(coord)

        if max(dist) > 10:
            continue

        param = coord.tolist()

        name = str(param).replace("[", "").replace("]", "").replace(",", "") 

        exampleData = {
            'type': types[mechType], 
            'params': param,
            'speedScale':speedscale, # 1 
            'steps':steps, # 360 
            'relativeTolerance':0.1 
        }


        batch.append(exampleData) #... You should use a function to decide save str. 
        batchSaveStr.append(saveDir + '/' + name )       # old method of doing things 
        batchSaveNpyStr.append(saveDirNpy + '/' + name ) # old method of doing things 

        if len(batch) >= batchCount:
            try:
                temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                time.sleep(0.02)
            except ValueError as v:
                for i in range(3):
                    time.sleep(2)
                    try:
                        temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                        break
                    except ValueError as v2:
                        errCtr += 1
            for i in range(len(temp)):
                P = np.array(temp[i]['poses'])
                try:
                    if len(P.shape) >= 1:
                        if P.shape[0] >= minsteps:
                            # do normalization, also get the transformation parameters. 
                            # also the paras are saved instead of MP (M: tranformation matrix, P: points in the matrix)
                            # This is just to avoid decimal difference problem 
                            imageMat, transParamSet, success = process_mech_051524(P, couplerCurveIndex[mechType])
                            if success:

                                # change Tstr if needed 
                            # normalized coords
                                normalized_coords = normalize_data_122223(P[:,couplerCurveIndex[mechType],:], scaling=3.5)[0]
                                #print(normalized_coords)
                                
                                '''plt.figure(figsize=(6, 6))
                                plt.plot(normalized_coords[:, 0], normalized_coords[:, 1], marker='o')
                                plt.title('Normalized Coupler Curve')
                                plt.xlabel('X')
                                plt.ylabel('Y')
                                plt.axis('equal')
                                plt.grid(True)
                                plt.show()'''

                                # Compute Fourier descriptors
                                #fd = np.fft.fft(normalized_coords[:, 0] + 1j * normalized_coords[:, 1])
                                #print(fd)

                                # Inverse FFT to reconstruct curve
                                #reconstructed = np.fft.ifft(fd)
                                #reconstructed_coords = np.column_stack((reconstructed.real, reconstructed.imag))

                                pca_output = detect_flat_via_pca(normalized_coords)

                                sharp, kappa = has_sharp_edges(normalized_coords, curvature_threshold=150.0)

                                self_intersecting = has_self_intersections(resample_uniform(normalized_coords, 512))
                                
                                #ground_clearance = ground_clearance_check(resample_uniform(P[:,couplerCurveIndex[mechType],:], 512), coord, pca_output['flat_indices'])
                                # batchSaveStr[i].replace(saveDir, '').replace('/', '') is a string of numbers separated by spaces
                                # Convert it to a list of coordinates (pairs)
                                #print("coordinates: " + batchSaveStr[i].replace(saveDir, '').replace('/', ''))
                                joint_coords = np.array(str_to_coords(batchSaveStr[i].replace(saveDir, '').replace('/', '')))
                                ground_clearance = ground_clearance_check(
                                    resample_uniform(P[:,couplerCurveIndex[mechType],:], 512),
                                    joint_coords,
                                    pca_output['flat_indices']
                                )
                                # resample uniform uses the unnormalized coords for everything
                                # P[:,couplerCurveIndex[mechType],:] is the coupler curve coords before normalization
                                # this needs resampling because pca_output does resampling to 512 points

                                # coord is the original joint coords in numPy array format
                                # pca_output['flat_indices'] are the indices of the flat portion of the coupler curve
                                if pca_output['is_flat_enough'] and is_closed(normalized_coords) and (not sharp) and (not self_intersecting):
                                    #plot_flat_fit_with_joints(resample_uniform(P[:,couplerCurveIndex[mechType],:], 512), pca_output['flat_indices'], joint_coords)
                                    #continue
                                    #print(name)
                                    pass
                                    #print("\033[95m" + f"flat but without ground clearance: \n mechtype: {types[mechType]} \n initla joints {param}, \n flat fraction {pca_output['flat_fraction']:.2f}, \nmax_run {pca_output['max_run']}, \nN {pca_output['N']}, \norientation {pca_output['orientation_deg']:.1f} deg, \nsharp edges {sharp}, ground clearance {ground_clearance}" + "\033[0m")

                                # if it satisfies everything and ground clearance, 
                                if pca_output['is_flat_enough'] and is_closed(normalized_coords) and (not sharp) and ground_clearance and (not self_intersecting):
                                
                                    #plot_flat_fit_with_joints(resample_uniform(P[:,couplerCurveIndex[mechType],:], 512), pca_output['flat_indices'], joint_coords)

                                    print("\033[92m" + f"flat but !!!!!WITH!!!!! ground clearance: \n mechtype: {types[mechType]} \n initla joints {param}, \n flat fraction {pca_output['flat_fraction']:.2f}, \nmax_run {pca_output['max_run']}, \nN {pca_output['N']}, \norientation {pca_output['orientation_deg']:.1f} deg, \nsharp edges {sharp}, ground clearance {ground_clearance}" + "\033[0m")

                                    #print("PCA Output:", pca_output)

                                    # change Tstr if needed 
                                    Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "") # Wei asked for this part
                                    Tstr = re.sub('\s+', ' ', Tstr) # to replace multiple sequential spaces together
                                    binary_data = np.uint8(imageMat[:,:,0]) * 255
                                    img = Image.fromarray(binary_data)

                                    img.save(saveDirWalking + "/" + name + ' ' + Tstr + '.jpg')
                                    save_flat_fit_with_joints(resample_uniform(P[:,couplerCurveIndex[mechType],:], 512), pca_output['flat_indices'], joint_coords, saveDirWalkingPlot, str(name + ' ' + Tstr + '.jpg'))
                                    print("img savbed to" + saveDirWalking + "/" + name + ' ' + Tstr + '.jpg')
                                    print("plot saved to" + str(name + ' ' + Tstr + '.jpg'))
                                    
                                    #plot_flat_fit_with_joints(resample_uniform(P[:,couplerCurveIndex[mechType],:], 512), pca_output['flat_indices'], batchSaveStr[i])

                                '''# Plot original and reconstructed curves
                                plt.figure(figsize=(6, 6))
                                plt.plot(normalized_coords[:, 0], normalized_coords[:, 1], label='Original', marker='o')
                                plt.plot(reconstructed_coords[:, 0], reconstructed_coords[:, 1], label='Reconstructed', linestyle='--', marker='x')

                                # Use only the first N low-frequency Fourier coefficients for smoothing
                                N = 10  # You can adjust N for more/less smoothing

                                fd_low = np.zeros_like(fd)
                                fd_low[:N] = fd[:N]
                                fd_low[-N+1:] = fd[-N+1:]

                                smoothed = np.fft.ifft(fd_low)
                                smoothed_coords = np.column_stack((smoothed.real, smoothed.imag))

                                plt.plot(smoothed_coords[:, 0], smoothed_coords[:, 1], label=f'Smoothed (N={N})', color='red', linewidth=2)



                                plt.title('Original vs Reconstructed Curve (FFT)')
                                plt.xlabel('X')
                                plt.ylabel('Y')
                                plt.axis('equal')
                                plt.grid(True)
                                plt.legend()
                                plt.show()

                                print(compute_fd_features(normalized_coords))


                                # change Tstr if needed 
                                Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "") # Wei asked for this part
                                Tstr = re.sub('\s+', ' ', Tstr) # to replace multiple sequential spaces together
                                binary_data = np.uint8(imageMat[:,:,0]) * 255
                                img = Image.fromarray(binary_data)
                                if needAddtional[mechType]: # need additional description (replace old method)
                                    pinit = (np.array(temp[i]['posInit'])[:savePointNumber[mechType], :].flatten().tolist())
                                    name = format_floats_to_string(pinit)
                                    #img.save(saveDir + '/' + name + ' ' + types[mechType] + ' '  + Tstr + '.jpg')
                                    # do not save all paths, only save walking
                                else:                       # traditional if no added 
                                    #img.save(batchSaveStr[i] + ' ' + types[mechType] + ' ' + Tstr + '.jpg')

                                if pca_output['is_flat_enough'] and is_closed(normalized_coords) and (not sharp):
                                    print("\033[94m" + f"found flat coupler curve: {batchSaveStr[i]} {types[mechType]} {Tstr}.jpg \n type: {types[mechType]}" + "\033[0m")
                                    print(coord)
                                #plt.imshow(imageMat)
                                #plt.savefig(batchSaveStr[i] + Tstr + '.jpg')
                                #plt.clf()'''
                            else: 
                                normFailCtr += 1
                            pass                        
                except ValueError as v:
                    print(v)
                except FileNotFoundError as f:
                    print(f)
            batch = []
            batchSaveStr = []
            batchSaveNpyStr = []

    if len(batch) >= batchCount:
        #print(batch[0], '\n', batch[1])
        #print(batchSaveStr[0], '\n', batchSaveStr[1])
        try:
            temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
            time.sleep(0.02)
        except ValueError as v:
            for i in range(3):
                time.sleep(2)
                try:
                    temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                    break
                except ValueError as v2:
                        errCtr += 1
        print(errCtr)
        for i in range(len(temp)):
            P = np.array(temp[i]['poses']) 
            try:
                if len(P.shape) >= 1:
                    if P.shape[0] >= minsteps:
                        # do normalization, also get the transformation parameters. 
                        # also the paras are saved instead of MP (M: tranformation matrix, P: points in the matrix)
                        # This is just to avoid decimal difference problem 
                        imageMat, transParamSet, success = process_mech_051524(P, couplerCurveIndex[mechType])
                        if success:
                            Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "") # Wei asked for this part
                            Tstr = re.sub('\s+', ' ', Tstr) # to replace multiple sequential spaces together
                            binary_data = np.uint8(imageMat[:,:,0]) * 255
                            img = Image.fromarray(binary_data)
                            if needAddtional[mechType]: # need additional description (replace old method)
                                pinit = (np.array(temp[i]['posInit'])[:savePointNumber[mechType], :].flatten().tolist())
                                name = format_floats_to_string(pinit)
                                #img.save(saveDir + '/' + name + ' ' + types[mechType] + ' '  + Tstr + '.jpg')
                            else:                       # traditional if no added 
                                #img.save(batchSaveStr[i] + ' ' + types[mechType] + ' '  + Tstr + '.jpg') 
                                continue

                            if pca_output['is_flat_enough'] and is_closed(normalized_coords) and (not sharp):
                                print("\033[94m" + f"found flat coupler curve: {batchSaveStr[i]} {types[mechType]} {Tstr}.jpg \n type: {types[mechType]}" + "\033[0m")
                            #plt.imshow(imageMat)
                            #plt.savefig(batchSaveStr[i] + Tstr + '.jpg')
                            #plt.clf()
                        else:
                            normFailCtr += 1
                            pass       
            except ValueError as v:
                print(v)
            except FileNotFoundError as f:
                print(f)
        batch = []
        batchSaveStr = []
        batchSaveNpyStr = []

RRRR


  0%|          | 861/500000 [00:03<30:45, 270.50it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/gpfs/software/Anaconda/envs/Jupyter/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2487570/3568655347.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/gpfs/software/Anaconda/envs/Jupyter/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2194, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/gpfs/software/Anaconda/envs/Jupyter/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1182, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/gpfs/software/Anaconda/envs/Jupyter/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1053, in structu

In [14]:
print(normFailCtr) # basically not too many steps is the reason. 
print(P.shape)

0
(360, 8, 2)
